In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.chat_models import ChatOpenAI
import os
import json

from langchain.memory import ConversationBufferMemory
from langchain_core.messages import (
    HumanMessage,
    SystemMessage
)
# import prompts
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
SUMMARY_TEMPLATE = """Generate song summary based on song data to provide song writter with short info about song. Respond ONLY with summary please."""

IDEAS_GENERATE_TEMPLATE = """As a seasoned scriptwriter for music videos, devise {ideas_number} creative concepts for a video project. Respond ONLY with python list of strings. each string is idea."""

CHOOSE_IDEA_TEMPLATE = """
You are professional video clip script writer.
Choose one of the ideas for video clip choose one that suites best . Respond ONLY with chosen idea. 
"""

SONG_DATA_TEMPLATE = """Here is song data:
Name: {name}
mood: {mood}
Artist: {artist}
Genre: {genre}
lyrics: {lyrics}
"""

FULL_SONG_DATA_TEMPLATE = """Here is song data:
Name: {name}
mood: {mood}
Artist: {artist}
Genre: {genre}
lyrics: {lyrics}
Summary: {summary}
"""

IDEAS_TEMPLATE = """Video Clip Ideas: {ideas}"""

IDEA_TEMPLATE = """Video Clip Idea: {idea}"""
SONG_SUMMARY_TEMPLATE = """Song summary: summary"""
CONCEPT_TEMPLATE = """Video Clip Concept: {concept}"""
SCENARIO_TEMPLATE = """Video Clip Scenario: {scenario}"""
CONCEPT_GENERATE_TEMPLATE = """As a seasoned scriptwriter for music videos. Based on song data and video clip idea generate 30 second video clip concept."""
SCENARIO_GENERATE_TEMPLATE = """As a seasoned scriptwriter for music videos. Based on song data and video clip idea+concept generate 30 second video clip scenario second by second."""


STORY_BOARD_GENERATE_TEMPLATE = """Write video clip storyboard by its info, describe each second. 
Respond with JSON ONLY no yapping where keys are seconds and values - second description. 
Describe clip from {start} second to {end} second"""

In [3]:
# Generate theme 
from dataclasses import dataclass

@dataclass
class Song:    
    name:str
    mood:str 
    artist:str
    genre:str
    lyrics:str
    summary:str=None

    def info_for_summarization(self):
        return {
            "name":self.name,
            "mood":self.mood,
            "artist":self.artist,
            "genre":self.genre,
            "lyrics":self.lyrics
        }
    
    def to_dict(self):
        return {
            "summary":self.summary,
            **self.info_for_summarization()
        }


@dataclass
class VideoClip:
    idea:str=None
    visual_aesthetic:str=None
    concept:str=None
    scenario:str=None
    story_board:str=None    


@dataclass 
class ScenarioWriter:
    name: str
    memory: ConversationBufferMemory
    chat: ChatOpenAI
    video_clip:VideoClip
    ideas:list=None

    def summarize_song(self, song:Song):
        self.song=song
        res = self.chat.invoke([
            SystemMessage(
                content=SUMMARY_TEMPLATE
            ),
            HumanMessage(
                content=SONG_DATA_TEMPLATE.format(**song.info_for_summarization())
            ),
        ])
        song.summary = res.content
        return res.content

    def idea_generation(self, ideas_number:int):
        res = self.chat.invoke([
            SystemMessage(
                content=IDEAS_GENERATE_TEMPLATE
            ),
            HumanMessage(
                content=SONG_DATA_TEMPLATE.format(**self.song.info_for_summarization())
            ),
            HumanMessage(
                content=SONG_SUMMARY_TEMPLATE.format(**{"summary":self.song.summary})
            ),
        ])
        # print(res.content)
        self.ideas=res.content
        return res.content

    
    def choose_best_idea(self, ideas):
        res = self.chat.invoke([
            SystemMessage(
                content=CHOOSE_IDEA_TEMPLATE
            ),
            HumanMessage(
                content=IDEAS_TEMPLATE.format(**{"ideas":ideas})
            ),
            HumanMessage(
                content=SONG_DATA_TEMPLATE.format(**self.song.to_dict())
            ),
        ])
        self.video_clip.idea = res.content
        return res.content
    
    def concept_development(self):
        res = self.chat.invoke([
            SystemMessage(
                content=CONCEPT_GENERATE_TEMPLATE
            ),
            HumanMessage(
                content=SONG_DATA_TEMPLATE.format(**self.song.to_dict())
            ),
            HumanMessage(
                content=IDEA_TEMPLATE.format(**{"idea":self.video_clip.idea})
            ),
            HumanMessage(
                content=CONCEPT_TEMPLATE.format(**{"concept":self.video_clip.concept})
            ),
        ])
        self.video_clip.concept = res.content
        return res.content

    # def visual_aesthetic():
    #     pass

    def write_scenario(self):
        res = self.chat.invoke([
            SystemMessage(
                content=SCENARIO_GENERATE_TEMPLATE
            ),
            HumanMessage(
                content=CONCEPT_TEMPLATE.format(**{"concept":self.video_clip.concept})
            ),
            HumanMessage(
                content=IDEA_TEMPLATE.format(**{"idea":self.video_clip.idea})
            ),
            HumanMessage(
                content=SONG_DATA_TEMPLATE.format(**self.song.to_dict())
            ),
        ])
        self.video_clip.scenario = res.content
        return res.content

    def create_story_board(self, start:int, end:int):
        messages = [
            SystemMessage(
                content=STORY_BOARD_GENERATE_TEMPLATE.format(**{'start':start, "end":end})
            ),
            HumanMessage(
                content=SCENARIO_GENERATE_TEMPLATE.format(**{"scenario":self.video_clip.scenario})
            ),
            HumanMessage(
                content=CONCEPT_TEMPLATE.format(**{"concept":self.video_clip.concept})
            ),
            HumanMessage(
                content=IDEA_TEMPLATE.format(**{"idea":self.video_clip.idea})
            ),
        ]
        res = self.chat.invoke(messages)

        self.video_clip.story_board = clean_text(res.content)

        return self.video_clip.story_board

def clean_text(input_text):
    # Define the substrings to be removed
    start_substring = "```json\n"
    end_substring = "```"
    
    # Remove the start substring
    if input_text.startswith(start_substring):
        input_text = input_text[len(start_substring):]
    
    # Remove the end substring
    if input_text.endswith(end_substring):
        input_text = input_text[:-len(end_substring)]
    
    return json.loads(input_text)


### Song data

In [4]:
name = "Space in Between"
mood = "Sad"
artist = "Jan Blomquist"
genre = "Techno"
lyrics = """
[' (Verse 1)\n'
 'He walks through the world with a heavy heart,\n'
 "A burden that's hidden, deep within his pants,\n"
 'His soul cries out for more than just sex,\n'
 'But the weight of his package, never gives him a chance.\n'
 '\n'
 '(Chorus)\n'
 'Oh, the man with the big dick,\n'
 'Feels so small and so sad,\n'
 "He's just a man with a heavy heart,\n"
 'And a cross that he never had.\n'
 '\n'
 '(Verse 2)\n'
 "He's tired of the stares, the whispers, the jokes,\n"
 'The expectations, the pressure, the constant remarks,\n'
 'He wants to be loved for who he is inside,\n'
 'But his big dick, always steals the spotlight, leaving him in the dark.\n'
 '\n'
 '(Chorus)\n'
 'Oh, the man with the big dick,\n'
 'Feels so alone and misunderstood,\n'
 "He's just a man with a heavy heart,\n"
 "Living in a world that's so crude.\n"
 '\n'
 '(Bridge)\n'
 "He's more than just a body, more than just a tool,\n"
 "He's a person with feelings, with dreams and with rules,\n"
 'He wants to find love, he wants to find peace,\n'
 'But his big dick, only brings him release.\n'
 '\n'
 '(Chorus)\n'
 'Oh, the man with the big dick,\n'
 'Feels so unloved and unseen,\n'
 "He's just a man with a heavy heart,\n"
 "Drowning in a world that's so mean.\n"
 '\n'
 '(Outro)
 "So here's to the man with the big dick,"
 'May he find love, may he find his way,
 'May he be seen for who he truly is,
 'And not just the size of his package, at the end of the day.
"""
openapi_api_key=os.getenv('OPENAPI_API_KEY')

### Inits

song = Song(
    name=name,
    mood=mood,
    artist=artist,
    genre=genre,
    lyrics=lyrics,
)
video_clip=VideoClip()
scenario_writer = ScenarioWriter(
    name="Oleg", 
    memory = ConversationBufferMemory(), 
    chat = ChatOpenAI(
        model_name='gpt-4-0125-preview', 
        openai_api_key=openapi_api_key, 
        temperature=0.7
    ),
    video_clip=video_clip,
)

In [6]:
summary = scenario_writer.summarize_song(song)

summary

'"Space in Between" by Jan Blomquist is a melancholic techno track that delves into the emotional turmoil of a man burdened by societal perceptions and expectations due to his physical attributes. The lyrics convey a sad narrative of isolation and the longing to be appreciated for one\'s true self, rather than being judged on superficial bases. Through a blend of somber verses and a poignant chorus, the song explores themes of loneliness, misunderstanding, and the desire for genuine connection and acceptance.'

In [7]:
ideas = scenario_writer.idea_generation(5)

ideas

'["A monochrome urban landscape, where the protagonist walks alone through crowded streets, feeling invisible despite his physical attribute being a source of unwanted attention.",\n "Scenes intercut between the protagonist facing awkward situations (dates, locker rooms) and him alone in a room, looking into a mirror with a mix of confusion and sadness.",\n "A symbolic sequence where the man is weighed down by literal chains representing societal expectations, struggling to move forward.",\n "A dreamlike sequence in a forest, where the protagonist is chasing after masked figures representing love and acceptance, but they keep disappearing into thin air.",\n "Close-up shots of different individuals, each showing signs of their own \'burdens\', implying that everyone has their own invisible struggles.",\n "A climactic scene where the protagonist stands at the edge of a cliff under the stars, contemplating the vastness of the universe, feeling both insignificant and liberated from societa

In [9]:
best_idea = scenario_writer.choose_best_idea(ideas)
best_idea

'Scenes intercut between the protagonist facing awkward situations (dates, locker rooms) and him alone in a room, looking into a mirror with a mix of confusion and sadness.'

In [10]:
concept = scenario_writer.concept_development()
concept

'**30-Second Video Clip Concept for "Space in Between" by Jan Blomquist**\n\n**Scene 1: [0:00-0:05]**\n- Close-up of a dimly lit room, focusing on the protagonist\'s eyes as he looks into a mirror. A single tear rolls down his cheek, reflecting the sad mood of the song. Soft, blue lighting accentuates his isolation.\n\n**Scene 2: [0:06-0:10]**\n- Flashback to a date, the protagonist and a woman sit across from each other at a cozy, candlelit table. The woman\'s eyes drop to his lap with a smirk, then back to his face, her demeanor changing to one of disinterest. The protagonist\'s smile fades, realizing the encounter is not what he hoped for.\n\n**Scene 3: [0:11-0:15]**\n- Cut to a locker room scene where the protagonist is awkwardly trying to change into his clothes while others around him sneak glances and whisper, highlighting the whispers and jokes mentioned in the lyrics. The lighting here is harsh and fluorescent, contrasting with the softer light of the previous intimate setting

In [11]:
scenario = scenario_writer.write_scenario()
scenario

'**Video Clip Scenario for "Space in Between" by Jan Blomquist**\n\n**[0:00-0:01]**\n- The video opens with a tight shot of the protagonist\'s face, reflecting a mix of confusion and sadness in the dim light, enhancing the song\'s melancholic mood.\n\n**[0:02-0:03]**\n- The scene quickly transitions to a flashback of an awkward date. The protagonist and a woman are at a dimly lit, intimate restaurant. She looks down with a smirk, then back up with disinterest.\n\n**[0:04-0:05]**\n- Cut back to the protagonist in front of the mirror, a single tear escapes as he reflects on the encounter, the room’s soft, blue lighting emphasizes his isolation.\n\n**[0:06-0:07]**\n- The scene shifts to the locker room, where the protagonist is trying to change. The atmosphere is tense, filled with whispers and sneering glances from others.\n\n**[0:08-0:09]**\n- A quick transition back to the room, the protagonist now sits on the floor, his back against the bed, looking lost and alone.\n\n**[0:10-0:11]**\

In [14]:
story_board = scenario_writer.create_story_board(0,31)
story_board

{'0': "Close-up of the protagonist's eyes in a dimly lit room, a tear rolling down his cheek.",
 '1': "The camera zooms out slightly, revealing the protagonist's reflection in the mirror.",
 '2': "Soft, blue lighting fills the room, highlighting the protagonist's isolation.",
 '3': 'Cut to a flashback of a candlelit table on a date.',
 '4': "The woman smirks, then shows disinterest, protagonist's smile fades.",
 '5': "Flashback ends, a quick transition back to the protagonist's eye in the mirror.",
 '6': 'Scene changes to a locker room, protagonist awkwardly changing.',
 '7': 'Others around him sneak glances and whisper.',
 '8': 'The fluorescent lighting contrasts with the previous intimate setting.',
 '9': "Cut back to the protagonist alone, the room's lighting dims further.",
 '10': 'The protagonist sits on the edge of his bed, head in hands.',
 '11': 'Close-up on his face, showing expressions of despair.',
 '12': 'The camera slowly zooms out, showing him alone in the room.',
 '13': 

In [18]:
res = {str(int(k)*16):v for k,v in story_board.items()}
with open('res.json', 'w') as file:
    json.dump(res, file, indent=4)